In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import os

# suppress warning
import warnings
warnings.filterwarnings('ignore')

import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
import scipy
from sklearn.metrics import r2_score
from statsmodels.graphics.tsaplots import plot_acf

#import seaborn as sns
#sns.set()

%matplotlib inline

In [20]:
# Get the current directory
current_directory = os.getcwd()

# Define the path to the data folder
data_folder = os.path.join(current_directory, 'data')

# List all files in the data folder
files = os.listdir(data_folder)

# Read each CSV file in the data folder
for file in files:
    if file.endswith('listings1.csv'):
        print("df1")
        file_path1 = os.path.join(data_folder, file)
        df1 = pd.read_csv(file_path1)
    elif file.endswith('listings2.csv'):
        print("df2")
        file_path2 = os.path.join(data_folder, file)
        df2 = pd.read_csv(file_path2)
    elif file.endswith("2021_NYC_Airbnb_Data.csv"):
        print("df2020")
        file_path2 = os.path.join(data_folder, file)
        df2 = pd.read_csv(file_path2)
        
        # Process the DataFrame df as needed

df1
df2


In [26]:
df1.size

2940150

In [25]:
df2.size

705636

In [30]:
df1.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [31]:
df2.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'license'],
      dtype='object')

In [27]:
df1[100:200]

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
100,893013997034954099,https://www.airbnb.com/rooms/893013997034954099,20240206005441,2024-02-06,city scrape,Rental unit in Queens · ★4.77 · 2 bedrooms · 3...,"Newly renovated, 20 minutes to Manhattan, two ...",NaN,https://a0.muscache.com/pictures/miso/Hosting-...,171807931,...,5.00,4.69,4.77,NaN,f,3,3,0,0,1.54
101,52842078,https://www.airbnb.com/rooms/52842078,20240206005441,2024-02-06,city scrape,Rental unit in Queens · 1 bedroom · 1 bed · 1 ...,Cozy space. Private Studio Apartment with full...,NaN,https://a0.muscache.com/pictures/5227b2ec-d456...,42024620,...,5.00,4.50,5.00,NaN,f,1,1,0,0,0.10
102,913059815216757944,https://www.airbnb.com/rooms/913059815216757944,20240206005441,2024-02-06,city scrape,Home in Queens · ★4.75 · 1 bedroom · 1 bed · 1...,Very quiet neighborhood 12 mins away from JFK ...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,92652420,...,4.88,4.50,4.88,NaN,f,1,1,0,0,1.46
103,24057583,https://www.airbnb.com/rooms/24057583,20240206005441,2024-02-06,city scrape,Rental unit in Staten Island · ★4.24 · 1 bedro...,Beautiful Studio Apartment with Private Entrance.,NaN,https://a0.muscache.com/pictures/d227fb8c-67e0...,181108819,...,4.72,4.48,4.33,NaN,f,1,1,0,0,1.32
104,556247091679540869,https://www.airbnb.com/rooms/556247091679540869,20240206005441,2024-02-06,city scrape,Rental unit in Queens · ★4.97 · 1 bedroom · 1 ...,Reconnect with loved ones in this family-frien...,"Located on a residential block, and only 2 blo...",https://a0.muscache.com/pictures/miso/Hosting-...,433944579,...,5.00,4.67,4.90,NaN,f,2,2,0,0,1.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,815287248948067069,https://www.airbnb.com/rooms/815287248948067069,20240206005441,2024-02-06,city scrape,Home in Brooklyn · ★New · 1 bedroom · 2 beds ·...,This unique place has a style all its own.,NaN,https://a0.muscache.com/pictures/d8d78ece-4a30...,498577349,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
196,781400979774367017,https://www.airbnb.com/rooms/781400979774367017,20240206005441,2024-02-06,city scrape,Rental unit in Brooklyn · 2 bedrooms · 3 beds ...,"Kick back and relax in this calm, stylish space.",NaN,https://a0.muscache.com/pictures/miso/Hosting-...,224047081,...,5.00,5.00,5.00,NaN,t,16,13,3,0,0.09
197,964420907082026117,https://www.airbnb.com/rooms/964420907082026117,20240206005441,2024-02-06,city scrape,Rental unit in The Bronx · 1 bedroom · 1 bed ·...,This unique place has a style all its own.,Steps to the Vancortlandt Park,https://a0.muscache.com/pictures/hosting/Hosti...,433014769,...,5.00,4.50,4.50,NaN,f,1,1,0,0,0.50
198,639222453616087462,https://www.airbnb.com/rooms/639222453616087462,20240206005441,2024-02-06,city scrape,Home in Queens · 1 bedroom · 1 bed · 1 shared ...,Rest at this peaceful and centrally-located pl...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,417806601,...,5.00,5.00,5.00,NaN,f,8,0,3,5,0.05


In [29]:
df2[100:200]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
100,995839463172093586,Rental unit in Brooklyn · ★New · 2 bedrooms · ...,517159189,Gerod,Brooklyn,Canarsie,40.644798,-73.907241,Entire home/apt,56.0,30,0,NaN,NaN,1,239,0,NaN
101,777572930730588568,Home in The Bronx · 1 bedroom · 1 bed · 1 priv...,474691192,Subhan,Bronx,Allerton,40.866614,-73.854978,Private room,500.0,30,0,NaN,NaN,1,364,0,NaN
102,1034089973112248847,Rental unit in Queens · 1 bedroom · 1 bed · 1 ...,548292248,John,Queens,Rosedale,40.662970,-73.733977,Private room,70.0,30,0,NaN,NaN,2,269,0,NaN
103,883896500051204160,Rental unit in New York · 1 bedroom · 1 bed · ...,317305683,Sebastian,Manhattan,Harlem,40.829619,-73.945204,Entire home/apt,76.0,30,0,NaN,NaN,1,286,0,NaN
104,1183005,Bed and breakfast in Brooklyn · 1 bedroom · 1 ...,6471326,Dawn,Brooklyn,East Flatbush,40.644836,-73.950409,Private room,89.0,30,1,2019-10-13,0.02,1,365,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,639222453616087462,Home in Queens · 1 bedroom · 1 bed · 1 shared ...,417806601,Seabelo,Queens,Jamaica Estates,40.719830,-73.790790,Shared room,48.0,30,1,2022-06-11,0.05,8,365,0,NaN
196,815287248948067069,Home in Brooklyn · ★New · 1 bedroom · 2 beds ·...,498577349,Daniel,Brooklyn,Flatlands,40.626130,-73.930380,Private room,60.0,30,0,NaN,NaN,1,89,0,NaN
197,51407096,Home in Staten Island · ★4.64 · 2 bedrooms · 4...,98868983,Nenita,Staten Island,Rosebank,40.616160,-74.069020,Shared room,152.0,30,33,2023-11-07,1.12,3,364,8,NaN
198,50172690,Rental unit in Brooklyn · ★4.60 · 1 bedroom · ...,220991097,Jessica,Brooklyn,Cypress Hills,40.679730,-73.882050,Private room,39.0,30,5,2024-01-03,0.17,3,123,4,NaN
